# AVALIAÇÃO INTERMEDIÁRIA DE SUPERCOMPUTAÇÃO

### ALUNA: LÍVIA SAYURI MAKUTA.

**Questão nro. 1 (4 pontos):**

Considere o problema de planejamento de roteiros para uma viagem de férias com amigos, em que cada dia é dedicado a uma atividade diferente e deve-se otimizar a ordem das atividades para minimizar o tempo total gasto em deslocamentos.
O problema consiste em dado um conjunto de atividades com localização, encontrar o melhor roteiro de viagem que minimize o tempo gasto em deslocamentos entre as atividades, considerando as limitações de
tempo e orçamento dos viajantes.

Para resolver esse problema, podemos usar uma heurística gulosa que funciona da seguinte maneira:

1. Escolha uma atividade inicial aleatória e adicione à lista de atividades a serem visitadas.
2. Para cada atividade não visitada, calcule a distância (ou tempo) para a atividade atualmente selecionada.
3. Selecione a atividade mais próxima da atual e adicione à lista de atividades a serem visitadas.
4. Repita o passo 2 e 3 até que todas as atividades sejam visitadas.
Sua tarefa: Você deve implementar essa heurística gulosa em C++ para encontrar o melhor roteirode viagem para um conjunto de atividades. A entrada do programa deve ser um arquivo com o seguinte
formato:
```
<número de atividades>
<latitude da atividade 1> <longitude da atividade 1>
<latitude da atividade 2> <longitude da atividade 2>
...
<latitude da atividade n> <longitude da atividade n>
```

O programa deve calcular a distância em metros entre duas localizações usando a seguinte função:

``` cpp
double distance(double lat1, double lon1, double lat2, double lon2) {
    const double R = 6371000;
    double phi1 = lat1 * M_PI / 180;
    double phi2 = lat2 * M_PI / 180;
    double dphi = (lat2 - lat1) * M_PI / 180;
    double dlambda = (lon2 - lon1) * M_PI / 180;
    double a = pow(sin(dphi / 2), 2) + cos(phi1) * cos(phi2) * pow(sin(dlambda / 2), 2);
    double c = 2 * atan2(sqrt(a), sqrt(1 - a));
    double distance = R * c;
    return distance;
}

```
Essa função recebe como parâmetros as latitudes e longitudes das duas localizações em graus e retorna a distância em metros. A constante M_PI representa o valor de pi em C++. A saída do programa deve ser o roteiro de viagem com as atividades em ordem e o distância total gasta em deslocamentos, em metros.

Exemplo: Suponha que temos um arquivo “atividades.txt” com o seguinte conteúdo:

```txt
5
-23.5505 -46.6333
-22.9068 -43.1729
-23.9658 -46.3331
-25.4439 -49.2709
-22.8758 -43.3143
```

A primeira linha indica que há 5 atividades. As próximas linhas representam as coordenadas geográficas de cada atividade. Seu programa deve produzir a seguinte saída:

Distância total percorrida: 1420941 metros

Ordem das atividades:

1 -> 3 -> 5 -> 2 -> 4

onde a ordem das atividades indica o roteiro de viagem otimizado.
No seu pacote de prova, você estará recebendo quatro arquivos de input para serem utilizados na resolução. Ao submeter a sua prova, você deve encaminhar, além do código-fonte, os arquivos de output, de modo que se o arquivo de input se chama questao1-input-10.txt, o arquivo de output deve ser questao1-output-10.txt. Você deve, portanto, encaminhar o output gerado pelo programa para cada
input fornecido.

**R:**

In [30]:
%%writefile exercicio1.cpp
#include <iostream>
#include <vector>
#include <cmath>
#include <algorithm> 
#include <random>
#include <chrono>
#include <stdlib.h> 
#include <fstream>
#include <iomanip>
#include <sstream>
#include <string> 
#include <string.h>
using std::vector;
using std::cin;
using std::cout;
using std::endl;

// Em relação a implementação, notei que a distância entre a última atividade e a primeira eram somadas, mas não fiz isso pois achei que não faz sentido.
// Ainda mais se estivermos pensando em limitações de custo, não faz sentido eu sair do último lugar no meu roteiro de atividades e voltar para o primeiro, 
// consumindo mais horas e na vida real, gastando mais com deslocamento.


double distance(double lat1, double lon1, double lat2, double lon2) {
    const double R = 6371000;
    double phi1 = lat1 * M_PI / 180;
    double phi2 = lat2 * M_PI / 180;
    double dphi = (lat2 - lat1) * M_PI / 180;
    double dlambda = (lon2 - lon1) * M_PI / 180;
    double a = pow(sin(dphi / 2), 2) + cos(phi1) * cos(phi2) * pow(sin(dlambda / 2), 2);
    double c = 2 * atan2(sqrt(a), sqrt(1 - a));
    double distance = R * c;
    return distance;
}


int main(){
    int n_cidades;
    cin >> n_cidades;

    vector<double> vazio (2, 0);
    vector<vector<double>> vetor_lat_long(n_cidades, vazio);

    for (int i = 0; i < n_cidades; i++){
        for (int j = 0; j < 2; j++){
            cin >> vetor_lat_long[i][j];
        }
    }

    vector<double> vazio_distancia (n_cidades, 0.0);
    vector<vector<double>> vetor_distancias(n_cidades, vazio_distancia);

    for (int i = 0; i < n_cidades; i++){
        for (int j = 0; j < n_cidades; j++){
            // aqui ele vai calcular a distancia em relacao a cidade fixada em i (vamos supor, cidade 0) com as demais que vao desde ela mesma - distancia 0, até todas as outras
            double distancia = distance(vetor_lat_long[i][0], vetor_lat_long[i][1], vetor_lat_long[j][0], vetor_lat_long[j][1]);
            vetor_distancias[i][j] = distancia;
        }
    }


    vector<int>ordem_atividades;


    unsigned seed = std::chrono::system_clock::now().time_since_epoch().count();
    std::default_random_engine generator (seed);
    std::uniform_int_distribution<int> distribution_distances(0, n_cidades-1);
    int primeira_atividade = distribution_distances(generator);

    ordem_atividades.push_back(primeira_atividade);

    double menor_distancia = 10000000000000.0;
    double distancia_total = 0;
    int atividade_escolhida = 0;

    for (int i = 1; i < n_cidades; i++){
        int cidade_analisada = ordem_atividades[i-1];
        for (int j = 0; j < n_cidades; j++){
            if (vetor_distancias[cidade_analisada][j] < menor_distancia && find(ordem_atividades.begin(), ordem_atividades.end(), j) == ordem_atividades.end()){
                menor_distancia = vetor_distancias[cidade_analisada][j];
                atividade_escolhida = j;
            }
        }

        ordem_atividades.push_back(atividade_escolhida);
        distancia_total+= menor_distancia;
        menor_distancia = 10000000000000.0;
    }

    std::string resultado = std::to_string(distancia_total);

    cout << "Distância total percorrida: " << resultado << " metros" << endl;
    cout << "Ordem das atividades: " << endl;
    for (int i = 0; i < n_cidades; i++){
        if (i == n_cidades-1){
            cout << ordem_atividades[i]+1;
        } else{
            cout << ordem_atividades[i]+1 << " -> ";
        }
    }


    return 0;
}



Overwriting exercicio1.cpp


In [31]:
!g++ -Wall -O3 -g exercicio1.cpp -o exercicio1

In [32]:
!./exercicio1 < atividades.txt

Distância total percorrida: 1081027.153127 metros
Ordem das atividades: 
1 -> 3 -> 5 -> 2 -> 4

Coloquei o código aqui só de exemplo, mas anexei o código os inputs e todos os outputs no Blackboard. Além disso, em relação a implementação, notei que a distância entre a última atividade e a primeira eram somadas, mas não fiz isso pois achei que não faz sentido. Ainda mais se estivermos pensando em limitações de custo, não faz sentido uma pessoa sair do último lugar no roteiro de atividades e voltar para o primeiro, isso só vai consumir mais horas e distância, e na vida real, iria acarretar mais gastos com deslocamento - como dito no enunciado, devemos considerar as limitações de tempo e orçamento dos viajantes.

**Questão nro. 2 (0,5 ponto):**

Com relação ao problema anterior, a estratégia gulosa nos garante uma
solução ótima? Justifique.

**R:** A heurística gulosa implementada anteriormente não garante uma solução ótima. Isso porque no problema anterior nós optamos por focar no exploitation, isto é, escolhemos uma propriedade para nos guiar durante a resolução do problema: no caso a menor distância entre as atividades, adotando a estratégia de começar escolhendo a primeira atividade de maneira aleatória e depois ir escolhendo as demais a partir da menor distância da atividade escolhida anteriormente. Nesse caso, estamos limitado nossa solução, principalmente quando escolhemos apenas a melhor solução local em cada etapa - estamos olhando apenas para a menor distância a partir da atividade já escolhida.

A heurística por si só, normalmente é uma técnica que prioriza a velocidade e que busca encontrar resultados aproximados que podem ou não ser os ótimos globais. Dessa forma, ela não garante que a solução será ótima, e como citado anteriormente, a cada passo ela escolhe a opção que parece ser a melhor entre as disponíveis naquele momento, sem considerar o impacto que essa escolha terá nas etapas futuras do processo. Ela apenas garante que em curto prazo soluções aproximadas e razoáveis serão encontradas, mas ela não vai considerar todas as possibilidades disponíveis e pode acabar ficando presa em um ótimo local.

No código anterior, por exemplo, a única parte que possui mais variedade é quando escolhemos a primeira atividade de maneira aleatória. Mas a partir daí, sempre que for escolhida aquela atividade como primeira, os resultados serão os mesmos, pois sempre estaremos priorizando a menor distância a partir do elemento anterior. 


**Questão nro. 3 (0,5 ponto):**

Proponha uma estratégia de busca exaustiva para resolução do problema da questão 1. Apresente um pseudo-código.

**R:** Para a questão 1, uma possível solução com busca exautiva seria:

- Primeiro, começar fazendo uma lista com todas as possíveis combinações de atividades. Ou seja, faríamos loops ou poderíamos utilizar algum algoritmo para gerar todas essas combinações (que seriam sempre armazenadas nessa lista).
- Com essa lista e todas as combinações, percorreríamos ela calculando a distância entre a localização de cada atividade para somar a distância total percorrida para cada combinação.
- E nessa iteração, teríamos uma variável `menor_distancia` que poderíamos definir inicialmente com um valor extremamente alto e que depois sempre compararíamos com a distância da combinação sendo analisada, e caso fosse menor, atualizaríamos a varíavel de menor distância. 
- Além disso, toda vez que essa variável fosse atualizada, teríamos uma outra variável (um vetor) para guardar a combinação que possui a menor distância até então.
- Por fim, com todas as iterações, obteríamos a menor distância e o vetor das atividades que seria a solução final.

**Questão nro. 4 (1 ponto):** 

Em sala de aula, nós implementamos diversas estratégias para a mochila binária. Explique a importância de buscar um balanço entre exploration e exploitation. Dê um exemplo de como buscamos atingir exploration e outro de como buscamos atingir exploitation no problema da mochila binária na estratégia de algoritmos genéticos.

**R:** Se focarmos apenas em *exploitation*, estaremos escolhendo uma propriedade em que vamos guiar a nossa solução e consequentemente limitando nosso conjunto de soluções que não vai ser todas as possíveis, mas sim apenas aquelas que a heurística a ser utilizada vai abranger, de modo que sempre serão produzidos os mesmos resultados - e pode acabar demorando para convergir para o valor ótimo. Por outro lado, se apenas focarmos em *exploration*, obteremos resultados aleatórios - o que pode ser qualquer uma das soluções possíveis e que consequentemente inclui resultados que não são soluções para o problema, mas que também pode gerar resultados que cheguem em um valor ótimo com menos passos. 

Assim, o balanceamento de *exploration* e *exploitation* é essencial. Pois dessa forma, posso explorar uma propriedade que creio ser importante para o meu problema, mas ao mesmo tempo trago uma certa aleatoriedade para não ficar preso em ótimos locais e variar a solução do meu problema. E foi isso que fizemos em aula quando, por exemplo, escolhemos que em 75% dos casos utilizaríamos a heurística do mais caro e que em 25% dos casos escolheríamos um item aleatório para entrar na mochila.

Sabendo dessa importância de balancear as estratégias, agora serão descrittos alguns exemplos de como podemos atingir *exploration* e *exploitation* para o problema da mochila binária com estratégia de algoritmos genéticos. Para a estratégia genética, primeiro teríamos que seguir alguns passos para assemelhar o algoritmo com as etapas de seleção natural, cruzamento e mutação. E alguns desses passos são:

1. Começar com uma população inicial de soluções aleatórias, como se fosse um cromossomo (a mochila seria então representada por bits). E já nesse primeiro passo poderíamos já optar por utilizar *exploration* (inicializando a mochila com itens aleatórios, por exemplo) ou *exploitation* (inicializando a mochila com alguns itens mais caros).

2. Feito isso cada mochila seria avaliada de acordo com seu valor e peso, para medir o quão boa é a solução, e já eliminar aquelas que não atendem os requisitos (fitness e selection). Aqui, poderíamos aplicar *exploitation*, buscando selecionar apenas mochilas mais caras (que poderiam ser possíveis soluções boas) ou mais leves (que permitem ser melhoradas nas próximas etapas) - dependendo do interesse. 

3. Com as mochilas selecionadas, elas passariam pelo cruzamento (crossover) para gerar mais mochilas. E esse cruzamento incluiria selecionar a parte de uma mochila com outra parte de outra mochila para gerar uma nova mochila. Para isso, poderíamos utilizar mais *exploitation*, juntando sempre a primeira parte de uma mochila com a última parte da mochila em sequência, por exemplo.

4. Por fim, a última etapa (de um ciclo que se repete inúmeras vezes), seria introduzir mutação para aumentar a diversidade das soluções geradas até então, o que busca evitar a convergência para soluções ótimas locais. Nesse caso, poderíamos utilizar *exploration* para inserir ou retirar itens aleatórios das mochilas já existentes. 





**Questão nro. 5 (1 ponto):**

No material complementar da prova, você encontra o arquivo `montecarlo.cpp`,
que calcula uma aproximação para o núumero π. Avalie o arquivo e, com o uso da ferramenta valgrind, execute o profiling do código, indicando quais trechos do código podem ser alvo de otimização. Para cada um destes pontos, descreva (não implemente) como você otimizaria estes pontos.

**R:** Primeiramente, rodei a compilação do programa com o seguinte comando que já inclui algumas otimizações:

``` bash
g++ -Wall -O3 -g montecarlo.cpp -o montecarlo
```

Depois rodei o seguinte comando para executar o valgrind:

``` bash
valgrind --tool=callgrind ./montecarlo
```

Feito isso foi gerado o seguinte documento: `callgrind.out.981109`. E para ver agora onde estavam os trechos de maior chamada de IR, foi executado o seguinte comando:

```bash
callgrind_annotate callgrind.out.981109 montecarlo.cpp
```

E esse comando retornou o seguinte:

``` txt

--------------------------------------------------------------------------------
Profile data file 'callgrind.out.981109' (creator: callgrind-3.15.0)
--------------------------------------------------------------------------------
I1 cache: 
D1 cache: 
LL cache: 
Timerange: Basic block 0 - 3652680
Trigger: Program termination
Profiled target:  ./montecarlo (PID 981109, part 1)
Events recorded:  Ir
Events shown:     Ir
Event sort order: Ir
Thresholds:       99
Include dirs:     
User annotated:   montecarlo.cpp
Auto-annotation:  off

--------------------------------------------------------------------------------
Ir         
--------------------------------------------------------------------------------
17,560,875  PROGRAM TOTALS

--------------------------------------------------------------------------------
Ir         file:function
--------------------------------------------------------------------------------
6,374,196  /build/glibc-SzIz7B/glibc-2.31/stdlib/random_r.c:random_r [/usr/lib/x86_64-linux-gnu/libc-2.31.so]
4,200,000  /build/glibc-SzIz7B/glibc-2.31/stdlib/random.c:random [/usr/lib/x86_64-linux-gnu/libc-2.31.so]
2,800,019  montecarlo.cpp:main [/home/user/Supercomp/PI/exercicio5/questao_05/montecarlo]
1,042,105  /build/glibc-SzIz7B/glibc-2.31/elf/dl-lookup.c:_dl_lookup_symbol_x [/usr/lib/x86_64-linux-gnu/ld-2.31.so]
1,000,000  /build/glibc-SzIz7B/glibc-2.31/stdlib/rand.c:rand [/usr/lib/x86_64-linux-gnu/libc-2.31.so]
  554,271  /build/glibc-SzIz7B/glibc-2.31/elf/dl-lookup.c:do_lookup_x [/usr/lib/x86_64-linux-gnu/ld-2.31.so]
  400,000  /build/glibc-SzIz7B/glibc-2.31/stdlib/../sysdeps/unix/sysv/linux/x86/lowlevellock.h:random
  400,000  ???:0x00000000001090a0 [???]
  273,638  /build/glibc-SzIz7B/glibc-2.31/elf/../sysdeps/x86_64/dl-machine.h:_dl_relocate_object
  117,185  /build/glibc-SzIz7B/glibc-2.31/elf/dl-lookup.c:check_match [/usr/lib/x86_64-linux-gnu/ld-2.31.so]
   90,133  /build/glibc-SzIz7B/glibc-2.31/elf/do-rel.h:_dl_relocate_object
   79,678  /build/glibc-SzIz7B/glibc-2.31/string/../sysdeps/x86_64/strcmp.S:strcmp [/usr/lib/x86_64-linux-gnu/ld-2.31.so]
   71,545  /build/glibc-SzIz7B/glibc-2.31/elf/dl-addr.c:_dl_addr [/usr/lib/x86_64-linux-gnu/libc-2.31.so]

--------------------------------------------------------------------------------
-- User-annotated source: montecarlo.cpp
--------------------------------------------------------------------------------
Ir        

        .  
        .  #include <time.h>
        .  #include <math.h>
        .  #include <iostream>
        .  
        .  #define ITERATIONS 100000
        .  
        4  int main(){
        .  
        1   int pi=0;
        5   srand((unsigned)time(NULL)); 
    6,817  => ???:0x00000000001090d0 (1x)
        8  => ???:0x00000000001090c0 (1x)
  200,000   for (int i=0;i<ITERATIONS; i++){
        .     double x,y;
1,000,000     x = (rand()%2) -1;
6,187,100  => ???:0x00000000001090a0 (100,000x)
  900,000     y = (rand()%2) -1;
6,187,096  => ???:0x00000000001090a0 (100,000x)
  400,000     if (x*x+y*y<=1)
  300,000        pi++;
        .   }
        .  
        4   std::cout<<pi*4.0/ITERATIONS;
        .  
        .   return 0;
        .  
        8  }

--------------------------------------------------------------------------------
Ir        
--------------------------------------------------------------------------------
2,800,022  events annotated

```

No código em si (desconsiderando as chamadas de bibliotecas externas), podemos ver alguns trechos que possuem muitas iterações e que poderiam ser otimizados:

- Primeiro, o número de iterações é muita grande, poderíamos dividir em threads a execução do programa.
- Segundo, estes trechos : `6,187,100  => ???:0x00000000001090a0 (100,000x)` aparecem logo depois da operação com o random ser efetuada, podendo indicar que algo aconteceu depois da chamada da função `rand()`. Uma mudança que pode ser feita para sortear muitos números seria : ao invés de utilizar o random, poderíamos utilizar a biblioteca `boost` - que como visto em aula é mais rápida e otimizada.
- Terceiro, para evitar a definição de variáveis que nem serão utilizadas para efetuar alguma operação posterior no código, ao invés de definí-las nesses trechos: `1,000,000     x = (rand()%2) -1;` e  `900,000     y = (rand()%2) -1;`, poderíamos simplesmente colocar essas definições diretamente no bloco do if. 
- Quarto, não entendi porque para as duas variáveis o valor sorteado foi subtraído em 1, isso porque os valores sorteados são ou 0 ou 1. Assim, quando o valor sorteado for subtraído em 1, os seus possíveis valores serão -1 e 0. Tanto é que depois essas variáveis são elevadas ao quadrado - provavelmente para pegar o resultado positivo. Isso é desnecessário bastava utilizar direto no if : if (rand()%2 + rand()%2 <= 1).



**Questão nro. 6 (3 pontos):**

 Em nosso projeto, buscamos desenvolver estratégias computacionais para
a alocação de filmes que podemos assistir, restrito a uma jornada de 24 horas, e respeitando a capacidade máxima de filmes por categoria. Há muitas formas de buscar resolver este problema e uma delas é por meio da abordagem de programação dinâmica. A programação dinâmica é uma abordagem computacional
que consiste em dividir um problema em subproblemas menores e resolver cada subproblema apenas uma vez, armazenando sua solução para uso futuro e evitando o retrabalho.

Sua tarefa: Implemente um algoritmo em C++ que resolva esse problema de nosso projeto utilizando programação dinâmica.

Vamos recordar como é a sua entrada, para que você possa compreender o pseudocódigo que estamos propondo para resolução: A entrada é composta por um arquivo texto chamado ”input.txt”. A primeira linha contém dois números inteiros separados por um espaço: N e M, respectivamente, representando o
número de filmes disponíveis e o número de categorias diferentes. A segunda linha contém M números inteiros separados por um espaço, representando o número máximo de filmes permitidos em cada categoria. As próximas N linhas contêm três números inteiros separados por um espaço: H, F e C, respectivamente,
representando o horário de início, horário de término e categoria do filme.

A seguir apresentamos o pseudocódigo proposto para resolução desse problema. Você deve basear sua implementação nesse pseudocódigo, fazer uso dos inputs gerados em seu projeto e discutir se está obtendo resultados melhores que os obtidos até o momento em suas implementações.

Não deixe de explicar como está funcionando a abordagem de programação dinâmica.

```txt
Ler N, M
Ler max_filmes[M]
Criar uma lista de filmes vazia

Para i de 1 at´e N
  Ler H[i], F[i], C[i]
  Adicionar o filme (H[i], F[i], C[i]) na lista de filmes

Ordenar a lista de filmes por ordem crescente de hora de término F[i]

Cria uma matriz dp[N+1][M+1] preenchida com zeros

Para j de 1 até M
  dp[0][j] = max_filmes[j]

Para i de 1 até N
  Para j de 1 até M
    Se C[i] != j
      dp[i][j] = dp[i-1][j]
    Senão
      last = i - 1
      Enquanto last >= 0 e F[last] > H[i]
        last = last - 1
      Fim Enquanto
      max_count = 0
      Para k de 0 até min(dp[last][j-1], F[i]-H[i]-1)
        max_count = max(max_count, dp[last][j-1-k] + k + 1)
      Fim Para
      dp[i][j] = max(dp[i-1][j], max_count)
    Fim Se
  Fim Para
Fim Para

Imprimir dp[N][M]

```
**R:**

In [26]:
%%writefile exercicio6.cpp
#include <iostream>
#include <vector>
#include <cmath>
#include <algorithm> 
#include <random>
#include <chrono>
#include <stdlib.h> 
#include <random>
#include <chrono>
#include <fstream>
#include <bitset>
#include <map>
#include <ctime>
using std::vector;
using std::cin;
using std::cout;
using std::endl;
using std::bitset;
using std::map;
using std::min;
using std::max;

struct Filme{
    int inicio;
    int fim;
    int categoria;
};

void ordena_final(vector<Filme> &vetor_filmes){
    std::sort(vetor_filmes.begin(), vetor_filmes.end(), [] (Filme &a, Filme &b){
		return a.fim < b.fim;
	});

}


// n é a qtd_filmes e m é o número de categorias

int main(){
    int n, m;
    cin >> n >> m;

    vector<int> max_filmes(m+1, 0);


    for (int i = 1; i < m+1; i++){
        cin >> max_filmes[i];
    }

    Filme filme_vazio = {0, 0, 0};
    vector<Filme> vetor_filmes (n+1, filme_vazio);

    for (int i = 1; i < n+1; i++){
        Filme filme;
        cin >> filme.inicio >> filme.fim >> filme.categoria;
        vetor_filmes[i] = filme;
    }


    ordena_final(vetor_filmes);

    // Cria uma matriz dp[N+1][M+1] preenchida com zeros
    vector<int> interno(m+1, 0);
    vector<vector<int>> dp(n+1, interno);

    for (int j = 1; j < m+1; j++){
        dp[0][j] = max_filmes[j];
    }


    for (int i = 1; i < n+1; i++){
        for (int j = 1; j < m+1; j++){
            if (vetor_filmes[i].categoria != j){
                dp[i][j] = dp[i-1][j];
            } else{
                int last = i - 1;
                while((last >= 0) && (vetor_filmes[last].fim > vetor_filmes[i].inicio)){ //enquanto  existe anterior e se o final do filme anterior é maior que o início do atual
                    last = last - 1;
                }
                int max_count = 0;
                for (int k = 0; k < min(dp[last][j-1], (vetor_filmes[i].fim - vetor_filmes[i].inicio - 1)); k++){
                    max_count = max(max_count, dp[last][j-1-k] + k + 1);
                }
                dp[i][j] = max(dp[i-1][j], max_count);
            }
        }
    }

    for (int i = 0; i < n+1; i++){
        for (int j = 0; j < m+1; j++){
            cout << dp[i][j] << " ";
        }
        cout << endl;
    }

    return 0;
}


Writing exercicio6.cpp


In [28]:
!g++ -Wall -O3 -g exercicio6.cpp -o exercicio6

In [29]:
!./exercicio6 < input1.txt

0 1 3 1 2 
0 1 3 4 2 
0 1 3 4 2 
0 1 3 4 2 
0 1 3 4 2 
0 1 3 4 2 
0 1 3 4 2 
0 1 3 4 2 
0 1 3 4 2 
0 1 3 4 2 
0 1 3 4 2 


Para comparar com o projeto, eu rodei o código do meu projeto para uma entrada com 10 filmes e 4 categorias (a mais simples primeiro) e depois com mil filmes e 100 categorias, e os resultados que eu obtive foram esses:

- Saída do projeto para quando testado para 10 filmes:

```
Foram vistos 2 filmes.
12 13 4
14 15 1
```

- Saída do projeto para quando testado para 1000 filmes:

```
Foram vistos 19 filmes.
1 1 8
2 3 1
3 4 9
4 6 7
7 7 6
8 9 3
9 10 6
10 11 5
11 12 4
12 13 2
13 14 6
14 16 4
17 18 9
18 19 10
19 20 2
20 21 1
21 22 1
22 23 5
24 24 4
```

Por sua vez, uma parte o resultado que obtive pelo vetor dp[n+1][m+1] foi o seguinte:

- Saída do código implementado para quando testado para 10 filmes e rodado anteriormente como exemplo:

```
1 3 1 2 
1 3 4 2 
1 3 4 2 
1 3 4 2 
1 3 4 2 
1 3 4 2 
1 3 4 2 
1 3 4 2 
1 3 4 2 
1 3 4 2 
1 3 4 2
```

- Saída do código implementado para quando testado para 1000 filmes (omiti a maior parte das saídas para não ficar muito grande):

```
4 5 8 9 7 9 3 1 5 1 
4 5 8 9 7 9 3 1 5 1 
4 5 8 9 7 9 3 1 5 1 
4 5 8 9 7 9 3 1 5 1 
4 5 8 9 7 9 3 4 5 1 
4 5 8 9 7 9 3 4 5 1 
4 5 8 9 7 9 3 4 5 1 
4 5 8 9 7 9 3 4 5 1 
4 5 8 9 7 9 3 4 5 1 
4 5 8 9 7 9 3 4 5 6 
4 5 8 9 7 9 3 11 5 6 
4 5 8 9 7 9 3 11 5 6 
4 5 8 9 10 9 3 11 5 6 
4 5 8 9 10 9 3 11 5 6 
4 5 8 9 10 9 10 11 12 6 
4 5 8 9 10 9 10 11 12 6 
4 5 8 9 10 11 10 11 12 6 
4 5 8 9 10 11 10 11 12 6 
4 5 8 9 10 11 10 11 12 6 
4 69 8 9 10 11 12 13 14 15 
4 69 8 9 10 11 12 13 14 15 
4 69 70 9 10 11 12 13 14 15 
4 69 70 9 10 11 12 13 14 15 
4 69 70 9 10 11 12 13 14 15 
4 69 70 9 10 11 12 13 14 15 
4 69 70 71 10 11 12 13 14 15 
4 69 70 71 10 11 12 13 14 15 
4 69 70 71 10 11 12 13 14 15 
4 69 70 71 10 11 12 13 14 15 
```

Como visto no pseudocódigo, a matriz dp é preenchida utilizando a abordagem de programação dinâmica, para exatamente não ter que recalcular os resultados, mas já aproveitá-los caso existam. A primeira linha da matriz (dp[0][j]) é preenchida com o número máximo de filmes permitidos em cada categoria. Em seguida, para cada filme i da lista, é verificado se ele pertence à categoria j. Se não pertencer, a solução para o subproblema é simplesmente a solução para o subproblema anterior (dp[i-1][j]). E com o `else` nós tentamos encontrar o último filme (last) que pertence àquela categoria j e que termina antes do início do filme i. A partir daí, é testado o número máximo de filmes que podem ser escolhidos considerando os filmes que foram escolhidos anteriormente (dp[last][j-1]) e o tempo disponível até o início do próximo filme (F[i]-H[i]-1). Depois, dp[i][j] é preenchido ou com o máximo obtido da solução anterior (dp[i-1][j]) ou com o número máximo de filmes que podem ser escolhidos.

Ou seja, pelo que entendi dessa implementação e comparando:
-  No projeto nós implementamos a heurística gulosa para descobrir quais filmes podem ser assistidos e sempre voltávamos a calcular quais filmes poderiam ser escolhidos - sempre preenchendo o bitset e sempre testando filmes que às vezes por conta do horário já sabíamos que não entrariam para solução, mas mesmo assim conferíamos.
-  Aqui, com esse implementação dinâmica, nós através de cálculos anteriores não precisamos calcular tudo novamente, reaproveitamos valores já obtidos - é por isso que a matriz vai sendo preenchida da esquerda para direita e de cima para baixo, tendo os resultados calculados no final da linha.  

EOF (End of file)